---------------------------------------------------------------------
# Twitter tweets - download all hashtag data
---------------------------------------------------------------------

### There are limitations in using Tweepy for scraping tweets. 
### The standard API only allows you to retrieve tweets up to 7 days ago 
### and is limited to scraping 18,000 tweets per a 15 minute window.


In [ ]:
import tweepy as tw
import pandas as pd 
import re
import pickle
from tqdm import tqdm



# Twitter API deatils
consumer_key= 'R4h8AHuGoIhHGaF8gqM9v7e09'
consumer_secret= '84uSwytHNXQ7uj9AxziMdrFzsdCE2VUsqk67JJ2nZGYcnUe09M'
access_token= '1130714178107400192-gsrzkNWAs3qhF0obWOgv251bEFh6AZ'
access_token_secret= 'oVqNwg56RmgJCrCCQHT7lOMjB1TBGDqAMmVZ1I3y0fuqz'

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)






In [ ]:
### ---- TODO---- #### ADD YOUR OWN PATH

# my_path = '/Users/Petra_Kummerova/Desktop/Python/NLP/Czechitas/'
# my_out_USE = "my_out_USE.csv"
# my_out_USE_all_columns = "my_out_USE_all_columns.csv"


### Initial data cleaning 

------------------------------------------------------------
#### Remove http and https link (find https until first space)
------------------------------------------------------------



In [ ]:
# clean tweets list for special characters
def clean_tweets(tweets_list_all):
    pattern = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    messages = [pattern.sub('', s) for s in tweets_list_all]
    return messages


### Tweets Download

In [ ]:
def tweets_dwnld(new_search,date_since):
    tweets = tw.Cursor(api.search,
                q=new_search,
                # geocode=geocodes,
                lang="en",
                since=date_since).items(1000)
    tweets_list = []
    tweets_authors = []
    tweets_geo = []
    tweets_date = []
    
    for tweet in tweets:
        tweets_list = tweets_list + [tweet.text]
        tweets_authors = tweets_authors + [tweet.user.name]
        tweets_geo = tweets_geo + [tweet.place]
        tweets_date = tweets_date + [tweet.created_at]
    # clean tweets for special characters
    messages = clean_tweets(tweets_list)
    df_tweets = pd.DataFrame({'topic': new_search, 'tweet': messages, 'author': tweets_authors, 'geo': tweets_geo, 'twdate': tweets_date})
    return df_tweets

def search_tweets(search_word_list, date_since):

    df_tweets_all = []
    for search_word in tqdm(search_word_list):
        print(f"Currently searching for {search_word}",end='\r')
        new_search = search_word + " -filter:retweets"
        df_tweets = tweets_dwnld(new_search,date_since)
        df_tweets_all.append(df_tweets)
      #  print("end of one loop")
    return df_tweets_all

In [ ]:
# Define the search term and the date_since date as variables
search_word_list = ["#Keboola","#python","#Tableau","#Snowflake","#R"]
geocodes = "50.0755, 14.4378°, 500km"
# Prague 50.0755° N, 14.4378° E
# search_word = "#Keboola" 
# new_search = search_word + " -filter:retweets"


date_since = "2021-08-20" # only goes back max around 7 days anyways..


In [ ]:
# takes approx. 7 mins, depending on internet connection and speed

df_tweets_all = search_tweets(search_word_list, date_since = date_since)


In [ ]:
df_tweets_all

#### Save into a pickle file.

In [ ]:

                                                                                                                                                                                                                   
pickle.dump( df_tweets_all, open( "df_tweets_all.p", "wb" ) )


#### Load the dictionary back from the pickle file.

In [ ]:

df_tweets_all_load = pickle.load( open( "df_tweets_all.p", "rb" ) )



#### Check one-by-one tweet categories (#Keboola","#python","#Tableau","#Snowflake","#R)

In [ ]:

print(df_tweets_all_load[0])
print(df_tweets_all_load[1])
print(df_tweets_all_load[2])
print(df_tweets_all_load[3])
print(df_tweets_all_load[4])


In [ ]:
df_tweets_all_load_joined = pd.concat(df_tweets_all_load, ignore_index=True)


In [ ]:
df_tweets_all_load_joined

In [ ]:
df_tweets_all_load_joined['tweet_ID'] = df_tweets_all_load_joined.index
df_tweets_all_load_joined.to_csv('df_tweets_all.csv')


In [ ]:
df_tweets_all_load_joined

In [ ]:
# takes the tweets from the dataframe
messages = df_tweets_all_load_joined["tweet"]

# Sentence Embeddings using USE 

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns





def generate_sentence_embeddings(sentences_list, module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"):
    
    embed = hub.load(module_url)
    sentence_embeddings = embed(sentences_list)
    
    return sentence_embeddings


In [ ]:
# might take up to 10 mins for first download/load, later on shall be ulpoaded within seconds from cache


### Generate embeddings for tweets

In [ ]:
message_embeddings = generate_sentence_embeddings(messages)

In [ ]:
print(message_embeddings)



#### Convert into numpy array

In [ ]:
message_embeddings = message_embeddings.numpy()

#### Save into a pickle file.




In [ ]:
pickle.dump( message_embeddings, open( "message_embeddings.p", "wb" ) )



#### Load the dictionary back from the pickle file.




In [ ]:
message_embeddings = pickle.load( open( "message_embeddings.p", "rb" ) )

# Clustering the embeddings 



### K-means clustering 


In [ ]:
import numpy as np

from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler

import pandas as pd

from sklearn.cluster import KMeans
import collections


In [ ]:
# convert message embeddings to DataFrame with numeric values

myX = pd.DataFrame(message_embeddings)
X = myX
X = X.apply(pd.to_numeric)



In [ ]:
# k-means clustering from text embeddings
# number of clusters to have on average 30 tweets in each - can be tested for more options / better fit 

n_clusters_proportionate = int(round(len(X)/30,0))

# apply k-means clustering 

kmeans_plus = KMeans(init='k-means++', n_clusters = n_clusters_proportionate, n_init=10)
kmeans_plus.fit(X)
cluster_labels = kmeans_plus.labels_

# join labels with predicted clusters

out = pd.DataFrame(columns = ['cluster_ID'])
out['cluster_ID'] = cluster_labels 
out_merged = pd.merge(out, df_tweets_all_load_joined, left_index=True, right_index=True) #  to check output clusters merged with raw data
print(out_merged)

# add tweet_ID to mapoing table as output for Tableau
out['tweet_ID'] = out.index 


In [ ]:
# save output to csv
out.to_csv(my_path + "out_cluster_twitter_ID_mapping.csv", sep = ';')





In [ ]:
# check some random clusters
print(out_merged[out_merged.cluster_ID == 5])



## TF-IDF to determine top topics within in each cluster

### Data preparation

In [ ]:
def tfidf_preproc(text):

    text = text.replace('.',' ')
    text = text.replace('#','')
    text = re.sub(r'\s+',' ',re.sub(r'[^\w \s]','',text) ).lower()

    return text

In [ ]:
out_merged['cleaned'] = [tfidf_preproc(i) for i in out_merged.tweet] 

In [ ]:
out_merged

#### Split the dataframe into clusters

In [ ]:
gb = out_merged.groupby('cluster_ID')  

In [ ]:

out_groups = [gb.get_group(x) for x in gb.groups]

In [ ]:
final = [''.join(i.cleaned) for i in out_groups]

In [ ]:
print(final[0])
print(final[20])

#### TF-IDF Vectorization 

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(final)
names = vectorizer.get_feature_names()
data = vectors.todense().tolist()
# Create a dataframe with the results
df = pd.DataFrame(data, columns=names)


In [ ]:
# nrows = no. clusters, ncols = no. words in tfidf, matrix values = tfidf of each word in each cluster

In [ ]:
df

### Remove all columns containing a stop word from the resultant dataframe. 

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
st = set(stopwords.words('english'))

df = df[filter(lambda x: x not in list(st) , df.columns)]

### Check top 10 topics within each cluster 

In [ ]:
def add_top_X_topics(df, X_topics):

    out_cluster_topics = pd.DataFrame(columns = ['tfidf', 'topics','cluster_ID'])

    n = X_topics;
    for i in df.iterrows():
        print(i[1].sort_values(ascending=False)[:n])
        my_df = pd.DataFrame(i[1].sort_values(ascending=False)[:n])
        cluster_id = list(my_df)[0]
        my_df['topics'] = my_df.index
        my_df.columns = ['tfidf', 'topics']
        my_df['cluster_ID'] = cluster_id
        out_cluster_topics = pd.concat([out_cluster_topics,my_df], ignore_index=True)

    return out_cluster_topics


### Join with full data to export to Tableau

In [ ]:

out_cluster_topics = add_top_X_topics(df,10)
out_cluster_topics.to_csv(my_path + 'out_cluster_metadata.csv', sep = ';')

# check out joined in master table with raw data
out_all_columns = pd.merge(out_merged, out_cluster_topics, right_on = 'cluster_ID', left_on = 'cluster_ID')


# Dimensionality reduction using PCA for cluster plotting 


In [ ]:
import numpy as np
from sklearn.decomposition import PCA


In [ ]:
def generate_pca_coordinates(original_vectors_array, num_components = 2):
    X = original_vectors_array
    pca = PCA(n_components=num_components)
    pca_embeddings = pca.fit_transform(X)
    pca_coords = pd.DataFrame(pca_embeddings,
                          columns=['x','y'])
    return pca_coords

In [ ]:
out_merged

In [ ]:
out_plot = out_merged.copy()

#### Reduce 512-dim to 2-dim to be able to visualize in 2D chart

In [ ]:
pca_coords = generate_pca_coordinates(message_embeddings, num_components=2)

In [ ]:
out_plot['x'] = pca_coords['x']

In [ ]:
out_plot['y'] = pca_coords['y']

In [ ]:
out_plot_pca = out_plot[['x','y','cluster_ID']]
out_plot_pca.to_csv(my_path + 'out_plot_pca.csv', sep = ';')

#### Create the 2D Visualization

In [ ]:

import matplotlib.pyplot as plt
import matplotlib.cm as cm  # to have nicer and more colors


N = len(out_plot)
x = out_plot['x']
y = out_plot['y']
colors = out_plot['cluster_ID']/100 # divide by 100 to have decimal numbners to have broader scale of colors

plt.scatter(x, y, c=cm.rainbow(colors), alpha=0.5)
plt.show()


# Hierarchical clustering

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage  
from matplotlib import pyplot as plt

#linked = linkage(np.array(X), 'single')
linked = linkage(np.array(X), 'ward')
labelList = out.index.values






In [ ]:
plt.figure(figsize=(10, 7))  
dendrogram(linked,  
            orientation='top',
            labels=labelList,
            distance_sort='descending',
            show_leaf_counts=True)
plt.show()  



# ---------- Tableau Input -------


### 1. Raw data (location, date, http link, twitter text)  + ID twitter + added hyperlinks with https = df_tweets_all.csv

### 2. ID twitter + ID cluster = out_cluster_twitter_ID_mapping.csv

### 3. ID cluster + Custer data (top 10 keywords) + ID keyword  = out_cluster_metadata.csv

### 4. ID cluster + x + y + ID Twitter (optional) = out_plot_pca.csv
